In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [209]:
#only use some records from sales table
def process_data(data_file, sale_random_number, category_score_level_list, product_unit_price_list):
    #load data
    def load_data(filename):
        df = pd.read_csv(filename, sep=";")
        return df

    catgDF = load_data(data_file+"/categories.csv")
    custDF = load_data(data_file+"/customers.csv")
    prodDF = load_data(data_file+"/products.csv")
    saleDF = load_data(data_file+"/sales.csv")
    
    #randomly select some sale data as sample
    sale_sampleDF = saleDF.sample(n=sale_random_number)
    
    #add some columns to some dataframe
    catgDF.insert(2, "ScoreLevel", category_score_level_list)
    custDF.insert(6, "Score", 0)
    custDF.insert(7, "LastPurchasingDate", None)
    custDF.insert(8, "FirstPurchasingDate", None)
    custDF.insert(9, "CustomerScoreLevel", 1.0)
    prodDF.insert(3, "UnitPrice", product_unit_price_list)
    
    sale_sampleDF["SalesDate"] = pd.to_datetime(sale_sampleDF["SalesDate"])
    custDF["LastPurchasingDate"] = pd.to_datetime(custDF["LastPurchasingDate"])
    custDF["FirstPurchasingDate"] = pd.to_datetime(custDF["FirstPurchasingDate"])
    
    return catgDF, custDF, prodDF, sale_sampleDF
    

In [226]:
#use all the records from sales table
def process_all_sale_data(data_file, category_score_level_list, product_unit_price_list):
    #load data
    def load_data(filename):
        df = pd.read_csv(filename, sep=";")
        return df

    catgDF = load_data(data_file+"/categories.csv")
    custDF = load_data(data_file+"/customers.csv")
    prodDF = load_data(data_file+"/products.csv")
    saleDF = load_data(data_file+"/sales.csv")
    
    #add some columns to some dataframe
    catgDF.insert(2, "ScoreLevel", category_score_level_list)
    custDF.insert(6, "Score", 0)
    custDF.insert(7, "LastPurchasingDate", None)
    custDF.insert(8, "FirstPurchasingDate", None)
    custDF.insert(9, "CustomerScoreLevel", 1.0)
    custDF.insert(10, "ExpectedPromotionDate", None)
    prodDF.insert(3, "UnitPrice", product_unit_price_list)
    
    saleDF["SalesDate"] = pd.to_datetime(saleDF["SalesDate"])
    custDF["ExpectedPromotionDate"] = pd.to_datetime(custDF["ExpectedPromotionDate"])
    custDF["LastPurchasingDate"] = pd.to_datetime(custDF["LastPurchasingDate"])
    custDF["FirstPurchasingDate"] = pd.to_datetime(custDF["FirstPurchasingDate"])
    
    
    return catgDF, custDF, prodDF, saleDF

In [124]:
#calculate customers' score
#def customer_score(sale_sampleDF, prodDF, catgDF, custDF):
#    def cal_score(row):
#        pid = row["ProductID"]
#       custid = row["CustomerID"]
#       cid = prodDF.loc[prodDF["ProductID"]==pid, "CategoryID"].values[0]
#       quantity = row["Quantity"]
#        score_level = catgDF.loc[catgDF["CategoryID"]==cid, "ScoreLevel"].values[0]
#        unit_price = prodDF.loc[prodDF["ProductID"]==pid, "UnitPrice"].values[0]
#        score = int(unit_price*quantity*score_level)
#        custDF.loc[custDF["CustomerID"]==custid, "Score"] += score
#        #set last purchasing date and first purchasing date
#        date = row["SalesDate"]
#        if(pd.isnull(custDF.at[custid, "LastPurchasingDate"])):
#            custDF.at[custid, "LastPurchasingDate"]= date
#        if(custDF.at[custid, "LastPurchasingDate"]<date):
#            custDF.at[custid, "LastPurchasingDate"] = date
#        if(pd.isnull(custDF.at[custid, "FirstPurchasingDate"])):
#            custDF.at[custid, "FirstPurchasingDate"] = date
#        if(custDF.at[custid, "FirstPurchasingDate"]>date):
#            custDF.at[custid, "FirstPurchasingDate"] = date
            
#    sale_sampleDF.apply(cal_score, axis=1)
#    return custDF


In [210]:
#calculate customers' score
def customer_score(sale_sampleDF, prodDF, catgDF, custDF):
    for index, row in sale_sampleDF.iterrows():
        pid = row["ProductID"]
        custid = row["CustomerID"]
        cid = prodDF.at[pid-1, "CategoryID"]
        quantity = row["Quantity"]
        date = row["SalesDate"]
        score_level = catgDF.at[cid-1, "ScoreLevel"]
        unit_price = prodDF.at[pid-1, "UnitPrice"]
        score = int(unit_price*quantity*score_level)
        custDF.at[custid-1, "Score"] += score
        #set last purchasing date and first purchasing date
        if(pd.isnull(custDF.at[custid-1, "LastPurchasingDate"])):
            custDF.at[custid-1, "LastPurchasingDate"]= date
        if(custDF.at[custid-1, "LastPurchasingDate"]<date):
            custDF.at[custid-1, "LastPurchasingDate"] = date
        if(pd.isnull(custDF.at[custid-1, "FirstPurchasingDate"])):
            custDF.at[custid-1, "FirstPurchasingDate"] = date
        if(custDF.at[custid-1, "FirstPurchasingDate"]>date):
            custDF.at[custid-1, "FirstPurchasingDate"] = date
        
    return custDF

In [272]:
#update"customer score level" column 
def update_cus_level(custDF):
    for index, row in custDF.iterrows():
        if(row["Score"]>50000):
            custDF.at[index, "CustomerScoreLevel"] = 1.5
        else:
            custDF.at[index, "CustomerScoreLevel"] = 1.0
    return custDF

In [212]:
#search customer's info
def search_cus(id):
    return custDF.loc[custDF["CustomerID"] == id]

In [213]:
#update customers' score and scorelevel with new transaction record
#this function has not been used yet since we only have historical data
#it's designed to update future transaction records
def update_cust_with_new_transaction(custDF, prodDF, catgDF, new_transactionDF):
    for index, row in new_transactionDF.iterrows():
        custid = row["CustomerID"]
        customerScoreLevel = custDF.at[custid-1, "CustomerScoreLevel"]
        quantity = row["Quantity"]
        pid = row["ProductID"]
        cid = prodDF.at[pid-1, "CategoryID"]
        date = row["SalesDate"]
        categoryScoreLevel = catgDF.at[cid-1, "ScoreLevel"]
        unit_price = prodDF.at[pid-1, "UnitPrice"]
        score = int(unit_price*quantity*categoryScoreLevel*customerScoreLevel)
        
        custDF.at[custid-1, "Score"] += score
    
    #after insert the new transaction records, update the customer's score level
    custDF = update_cus_level(custDF)
    return custDF

In [214]:
#use some sale records to test

#make a category score level list
test_score_level_list = np.random.randint(5, high=10, size=11)
#print test_score_level_list
test_score_level_list = [float(i)/10 for i in test_score_level_list]
#print test_score_level_list

#make a product unit price list
test_unit_price_list = np.random.randint(5, high=100, size=452)

catgDF, custDF, prodDF, sale_sampleDF = process_data(data_file="data", sale_random_number=100000, category_score_level_list=test_score_level_list, product_unit_price_list=test_unit_price_list)
new_custDF = customer_score(sale_sampleDF=sale_sampleDF, prodDF=prodDF, catgDF=catgDF, custDF=custDF)

#timestamp to string 
new_custDF["LastPurchasingDate"] = new_custDF["LastPurchasingDate"].astype(str)
new_custDF["FirstPurchasingDate"] = new_custDF["FirstPurchasingDate"].astype(str)

#update customer's score level
new_custDF = update_cus_level(new_custDF)

#export to a json file
new_custDF[new_custDF["Score"]!=0].to_json(path_or_buf="test.json", orient="records")


In [221]:
new_custDF[new_custDF["CustomerScoreLevel"]!=1].head()

,CustomerID,FirstName,MiddleInitial,LastName,CityID,Address,Score,LastPurchasingDate,FirstPurchasingDate,CustomerScoreLevel
24652,24653,Marcos,J,Cain,33,174 Fabien Boulevard,2194,2018-04-29 15:34:38.200,2018-01-04 02:38:37.490,1.5
25082,25083,Sherman,J,Hancock,26,19 New Avenue,1776,2018-03-26 06:55:16.580,2018-02-07 16:11:34.070,1.5
25186,25187,Guillermo,A,Davenport,60,640 White Second Freeway,1583,2018-05-02 11:39:40.510,2018-01-13 19:25:49.880,1.5
27750,27751,Sandra,C,Mayer,92,209 Milton Parkway,1608,2018-04-27 18:20:07.400,2018-01-01 04:57:05.190,1.5
27780,27781,Joe,S,Carney,65,185 North Hague Boulevard,1622,2018-03-18 03:19:03.070,2018-01-18 01:48:41.810,1.5


In [217]:
new_custDF.dtypes

CustomerID               int64
FirstName               object
MiddleInitial           object
LastName                object
CityID                   int64
Address                 object
Score                    int64
LastPurchasingDate      object
FirstPurchasingDate     object
CustomerScoreLevel     float64
dtype: object

In [222]:
search_cus(24652)

,CustomerID,FirstName,MiddleInitial,LastName,CityID,Address,Score,LastPurchasingDate,FirstPurchasingDate,CustomerScoreLevel
24651,24652,Alicia,F,Doyle,91,58 West Milton Parkway,390,2018-04-30 18:08:01.410,2018-04-30 18:08:01.410,1.0


In [228]:
#use all the data 


catgDF_all, custDF_all, prodDF_all, saleDF_all = process_all_sale_data(data_file="data", category_score_level_list=test_score_level_list, product_unit_price_list=test_unit_price_list)
new_custDF_all = customer_score(sale_sampleDF=saleDF_all, prodDF=prodDF_all, catgDF=catgDF_all, custDF=custDF_all)

#update customer's score level
new_custDF_all = update_cus_level(new_custDF_all)


In [229]:
new_custDF_all.head()

,CustomerID,FirstName,MiddleInitial,LastName,CityID,Address,Score,LastPurchasingDate,FirstPurchasingDate,CustomerScoreLevel,ExpectedPromotionDate
0,1,Stefanie,Y,Frye,79,97 Oak Avenue,2199,2018-05-06 10:19:40.780,2018-01-03 05:24:59.690,1.5,NaT
1,2,Sandy,T,Kirby,96,52 White First Freeway,2142,2018-05-04 21:59:55.310,2018-01-01 09:25:58.360,1.5,NaT
2,3,Lee,T,Zhang,55,921 White Fabien Avenue,2313,2018-05-09 02:30:55.190,2018-01-01 06:32:06.410,1.5,NaT
3,4,Regina,S,Avery,40,75 Old Avenue,2309,2018-05-08 11:33:36.170,2018-01-03 00:36:59.240,1.5,NaT
4,5,Daniel,S,Mccann,2,283 South Green Hague Avenue,1886,2018-05-09 11:05:35.920,2018-01-02 08:08:29.000,1.5,NaT


In [230]:
new_custDF_all.shape

(98759, 11)

In [269]:
new_custDF_all[["CustomerID", "CustomerScoreLevel"]].to_csv(path_or_buf="customerLevel.csv", index=False)

In [256]:
custIntervalDF = pd.read_csv("customerInterval.csv")

In [257]:
custIntervalDF.head()

,CustomerID,Interval
0,1,3
1,2,3
2,3,1
3,4,2
4,5,2


In [258]:
new_custDF1 = pd.merge(new_custDF_all, custIntervalDF, on="CustomerID")

In [259]:
new_custDF1.head()

,CustomerID,FirstName,MiddleInitial,LastName,CityID,Address,Score,LastPurchasingDate,FirstPurchasingDate,CustomerScoreLevel,ExpectedPromotionDate,Interval
0,1,Stefanie,Y,Frye,79,97 Oak Avenue,2199,2018-05-06 10:19:40.780,2018-01-03 05:24:59.690,1.5,NaT,3
1,2,Sandy,T,Kirby,96,52 White First Freeway,2142,2018-05-04 21:59:55.310,2018-01-01 09:25:58.360,1.5,NaT,3
2,3,Lee,T,Zhang,55,921 White Fabien Avenue,2313,2018-05-09 02:30:55.190,2018-01-01 06:32:06.410,1.5,NaT,1
3,4,Regina,S,Avery,40,75 Old Avenue,2309,2018-05-08 11:33:36.170,2018-01-03 00:36:59.240,1.5,NaT,2
4,5,Daniel,S,Mccann,2,283 South Green Hague Avenue,1886,2018-05-09 11:05:35.920,2018-01-02 08:08:29.000,1.5,NaT,2


In [260]:
new_custDF1.dtypes

CustomerID                        int64
FirstName                        object
MiddleInitial                    object
LastName                         object
CityID                            int64
Address                          object
Score                             int64
LastPurchasingDate       datetime64[ns]
FirstPurchasingDate      datetime64[ns]
CustomerScoreLevel              float64
ExpectedPromotionDate    datetime64[ns]
Interval                          int64
dtype: object

In [261]:
new_custDF1["LastPurchasingDate"] = pd.to_datetime(new_custDF1["LastPurchasingDate"])

In [262]:
new_custDF1["LastPurchasingDate"] = new_custDF1["LastPurchasingDate"].dt.date
new_custDF1["LastPurchasingDate"] = pd.to_datetime(new_custDF1["LastPurchasingDate"], format="%Y-%m-%d")

new_custDF1["FirstPurchasingDate"] = new_custDF1["FirstPurchasingDate"].dt.date
new_custDF1["FirstPurchasingDate"] = pd.to_datetime(new_custDF1["FirstPurchasingDate"], format="%Y-%m-%d")

In [263]:
new_custDF1.head()

,CustomerID,FirstName,MiddleInitial,LastName,CityID,Address,Score,LastPurchasingDate,FirstPurchasingDate,CustomerScoreLevel,ExpectedPromotionDate,Interval
0,1,Stefanie,Y,Frye,79,97 Oak Avenue,2199,2018-05-06,2018-01-03,1.5,NaT,3
1,2,Sandy,T,Kirby,96,52 White First Freeway,2142,2018-05-04,2018-01-01,1.5,NaT,3
2,3,Lee,T,Zhang,55,921 White Fabien Avenue,2313,2018-05-09,2018-01-01,1.5,NaT,1
3,4,Regina,S,Avery,40,75 Old Avenue,2309,2018-05-08,2018-01-03,1.5,NaT,2
4,5,Daniel,S,Mccann,2,283 South Green Hague Avenue,1886,2018-05-09,2018-01-02,1.5,NaT,2


In [255]:
#find today. Since the data is historical, so we define today as the last day in the sale data
saleDF_all["SalesDate"].max()

Timestamp('2018-05-09 23:59:59.400000')

In [266]:
from datetime import timedelta
new_custDF1.at[0, "LastPurchasingDate"] + timedelta(new_custDF1.at[0, "Interval"])

Timestamp('2018-05-09 00:00:00')

In [267]:
for index, row in new_custDF1.iterrows():
    new_custDF1.at[index, "ExpectedPromotionDate"] = row["LastPurchasingDate"] + timedelta(row["Interval"])

In [268]:
new_custDF1.head()

,CustomerID,FirstName,MiddleInitial,LastName,CityID,Address,Score,LastPurchasingDate,FirstPurchasingDate,CustomerScoreLevel,ExpectedPromotionDate,Interval
0,1,Stefanie,Y,Frye,79,97 Oak Avenue,2199,2018-05-06,2018-01-03,1.5,2018-05-09,3
1,2,Sandy,T,Kirby,96,52 White First Freeway,2142,2018-05-04,2018-01-01,1.5,2018-05-07,3
2,3,Lee,T,Zhang,55,921 White Fabien Avenue,2313,2018-05-09,2018-01-01,1.5,2018-05-10,1
3,4,Regina,S,Avery,40,75 Old Avenue,2309,2018-05-08,2018-01-03,1.5,2018-05-10,2
4,5,Daniel,S,Mccann,2,283 South Green Hague Avenue,1886,2018-05-09,2018-01-02,1.5,2018-05-11,2


In [271]:
new_custDF1.to_csv(path_or_buf="custInfo.csv", index=False)

In [273]:
new_custDF2 = update_cus_level(new_custDF1)

In [274]:
new_custDF2.to_csv(path_or_buf="custInfo.csv", index=False)

In [2]:
testDF = pd.read_csv("custInfo.csv")

In [3]:
testDF.head()

,CustomerID,FirstName,MiddleInitial,LastName,CityID,Address,Score,LastPurchasingDate,FirstPurchasingDate,CustomerScoreLevel,ExpectedPromotionDate,Interval
0,1,Stefanie,Y,Frye,79,97 Oak Avenue,2199,2018-05-06,2018-01-03,1.0,2018-05-09,3
1,2,Sandy,T,Kirby,96,52 White First Freeway,2142,2018-05-04,2018-01-01,1.0,2018-05-07,3
2,3,Lee,T,Zhang,55,921 White Fabien Avenue,2313,2018-05-09,2018-01-01,1.0,2018-05-10,1
3,4,Regina,S,Avery,40,75 Old Avenue,2309,2018-05-08,2018-01-03,1.0,2018-05-10,2
4,5,Daniel,S,Mccann,2,283 South Green Hague Avenue,1886,2018-05-09,2018-01-02,1.0,2018-05-11,2


In [5]:
testDF[["CustomerID", "CustomerScoreLevel"]].to_csv(path_or_buf="customerLevel.csv", index=False)